# Advent of Code 2021 - Day 11

In [21]:
typealias Octopuses = List<List<Int>>
typealias Index = Pair<Int, Int>
val ENERGY_RANGE = (0..9)

In [22]:
import java.io.File

val octopuses: List<List<Int>> = File("Day11.input.txt")
  .bufferedReader()
  .readLines()
  .map { row -> row.map(Char::digitToInt) }

## Part 1

Simulate the `octopuses` over the course of multiple steps. In a single step the following occurs:
- Increase the `energy` of all `octopuses` by 1.
- If any octopus has an energy level greater than 9, increase the energy level of all adjacent (including diagonals) octopus by 1. This repeats until no other octopus is flashing. Each octopus can only flash once per step.
- Any octopus that flashed previously sets its `energy` level to 0.
Count the total number of flashes after 100 steps.

In [23]:
operator fun List<List<*>>.contains(index: Index): Boolean = index.first in indices && index.second in this[index.first].indices

fun Octopuses.map2D(fn: (Int) -> Int): Octopuses = map { row -> row.map { fn(it) } }

fun <T> Octopuses.foldIndexed2D(acc: T, fn: (Index, T, Int) -> T): T = 
    foldIndexed(acc) { r, outer, row -> 
        row.foldIndexed(outer) { c, inner, it -> fn(r to c, inner, it) } 
    }

fun Octopuses.powerUp(): Octopuses = map2D { it + 1 }

fun Octopuses.exhaust(): Octopuses = map2D { if (it > ENERGY_RANGE.endInclusive) ENERGY_RANGE.start else it }

fun Octopuses.flash(): Pair<Octopuses, Set<Index>> {
    val completed: MutableSet<Index> = mutableSetOf()
    val processing = flashing().toMutableSet()
    val state: MutableList<MutableList<Int>> = map { it.toMutableList() }.toMutableList()

    while (processing.isNotEmpty()) {
        processing.first().also{ processing.remove(it) }.let { index -> 
            completed.add(index)
            flashRange(index).onEach { inRange ->
                state[inRange.first][inRange.second] += 1
                processing.addAll(state.flashing().filter { it !in completed })
            }
        }
    }

    return state to completed
}

fun Octopuses.flashing(): Set<Index> = foldIndexed2D(mutableSetOf()) { index, acc, it ->
    if (it > ENERGY_RANGE.endInclusive) acc.apply { add(index) }
    else acc
}

fun Octopuses.flashRange(index: Index): List<Index> = (-1..1)
    .flatMap { row -> (-1..1).map { col -> (index.first + row) to (index.second + col) } }
    .filter { it in this && it != index }

tailrec fun Octopuses.step(times: Int = 1, flashes: Int = 0): Pair<Octopuses, Int> = 
    if (times == 0) this to flashes
    else powerUp().flash().let {
        it.first.exhaust().step(times - 1, flashes + it.second.size)
    }

octopuses.step(100).second

1700

### Notes

The complex part of this problem is related to the `flashing` logic. We need to keep track of which index needs to flash and which have already flashed (in order to not double up) on flashing. To do this, we can keep track of the indices via a `Set`. I couldn't really figure out a good way to implement `Octopuses::flash` without keeping a copy of the `state` and the `completed` / `processing` sets.

## Part 2

At what step do all `octopuses` flash?

In [24]:
fun Octopuses.allFlashing() = all { row -> row.all { it == 0 } }

fun Octopuses.stepsUntil(condition: (Octopuses) -> Boolean): Int {
    var octopuses = this
    var count = 0
    while (!condition(octopuses)) {
        octopuses = octopuses.step().first
        count++
    }
    return count
}

octopuses.stepsUntil { it.allFlashing() }

273

### Notes

We can reuse most of the logic from part 1 and just step until all the `octopuses` are flashing. I think a cleaner way to do this might have been with an infinite `generateSequence` but couldn't figure it out. Ideally the sequence would keep stepping from the previous element.